## Conectarse a la base de datos

Para conectar la base de datos a este notebook, se utilizará las siguientes librerias:

1. xlrd
2. sqlalchemy
3. pandas

Las versiones se encuentran en el documento requirements.

Se importa estas librerias para poder conectarse, por razones de privacidad no subiré la conectividad para crear la base de datos

In [ ]:
import xlrd
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("#####")

Para crear la base de datos
Se van a crear 4 variables:
1. Donde 1 será para conectarme a la base de datos.
2. Y los demás serán las tablas.

In [ ]:
from sqlalchemy import text

nombre_base_datos = "sandbox"
nombre_tabla_ventas = "jpra_ventas"
nombre_tabla_calendario = "jpra_calendario"
nombre_tabla_pedidos = "jpra_pedidos"

with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {nombre_base_datos}"))
    conn.execute(text(f"USE {nombre_base_datos}"))

In [ ]:
with engine.connect() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {nombre_tabla_ventas}"))
    conn.execute(text(f"""
        CREATE TABLE IF NOT EXISTS {nombre_tabla_ventas} (
            FAMILIA VARCHAR(255),
            Tipo VARCHAR(255),
            FechaVenta DATE,
            HoraVenta INT,
            Articulo VARCHAR(255),
            Cantidad FLOAT,
            Precio FLOAT,
            Importe FLOAT
        )
    """))
    conn.commit() 

In [ ]:
with engine.connect() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {nombre_tabla_calendario}"))
    conn.execute(text(f"""
        CREATE TABLE IF NOT EXISTS {nombre_tabla_calendario} (
            Fecha DATE,
            Festivo VARCHAR(255)
        )
    """))
    conn.commit()

In [ ]:
with engine.connect() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {nombre_tabla_pedidos}"))
    conn.execute(text(f"""
        CREATE TABLE IF NOT EXISTS {nombre_tabla_pedidos} (
            Tipo VARCHAR(255),
            Fecha DATE,
            Articulo VARCHAR(255),
            Cantidad FLOAT,
            Precio FLOAT,
            Importe FLOAT
        )
    """))
    conn.commit()

Se asignan las rutas de los documentos en varibales para subir a Dbeaver

In [ ]:
DATA_PATH = '../data/dathaton/'
nombre_archivo_ventas = DATA_PATH + 'ArticulosPanaderia.xlsx'

df = pd.read_excel(nombre_archivo_ventas)

registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = text(f"""
    INSERT INTO {nombre_tabla_ventas} 
    (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) 
    VALUES (:familia, :tipo, :fecha_venta, :hora_venta, :articulo, :cantidad, :precio, :importe)
""")

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
with engine.connect() as connection:
    for i in range(0, len(registros), tamano_lote):
        lote_registros = registros[i:i + tamano_lote]
        params = [
            {
                "familia": r[0], "tipo": r[1], "fecha_venta": r[2], "hora_venta": r[3],
                "articulo": r[4], "cantidad": r[5], "precio": r[6], "importe": r[7]
            }
            for r in lote_registros
        ]
        connection.execute(consulta, params)

In [ ]:
nombre_archivo_calendario = DATA_PATH + 'Calendario.xlsx'

df = pd.read_excel(nombre_archivo_calendario)

registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = text(f"""
    INSERT INTO {nombre_tabla_calendario} 
    (Fecha, Festivo) 
    VALUES (:fecha, :festivo)
""")

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
with engine.connect() as connection:
    for i in range(0, len(registros), tamano_lote):
        lote_registros = registros[i:i + tamano_lote]
        params = [
            {
                "fecha": r[0], "festivo": r[1]
            }
            for r in lote_registros
        ]
        connection.execute(consulta, params)
    connection.commit()

In [ ]:
nombre_archivo_pedidos = DATA_PATH + 'CantidadPedida.xlsx'

df = pd.read_excel(nombre_archivo_pedidos)

registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = text(f"""
    INSERT INTO {nombre_tabla_pedidos} 
    (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) 
    VALUES (:tipo, :fecha, :articulo, :cantidad, :precio, :importe)
""")

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
with engine.connect() as connection:
    for i in range(0, len(registros), tamano_lote):
        lote_registros = registros[i:i + tamano_lote]
        params = [
            {
                "tipo": r[0], "fecha": r[1], "articulo": r[2], "cantidad": r[3],
                "precio": r[4], "importe": r[5]
            }
            for r in lote_registros
        ]
        connection.execute(consulta, params)
    connection.commit()

Nota: las consultas siguientes están preparadas para lanzarse con los datos cargados en la bbdd data y con los nombres iniciales.

In [ ]:
consulta_proceso="""

/*Ordenamos los productos por familia*/
drop table if exists sandbox.jpra_articulos_top;
create table sandbox.jpra_articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from sandbox.jpra_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Creamos un calendario general del 1/1/2017 al 31/12/2023*/
drop table if exists sandbox.jpra_calendario_dias;
create table sandbox.jpra_calendario_dias as (

with recursive jpra_cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from jpra_cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_date,'%x-%v') as semana -- formato
from jpra_cte_calendario

);

/*Añadimos al calendario los festivos*/
drop table if exists sandbox.jpra_calendario_completo;
create table sandbox.jpra_calendario_completo as (
select
base.*,
festivos.festivo
from sandbox.jpra_calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from sandbox.jpra_calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);

/*Agregamos las ventas por día (quitamos el detalle por hora)*/

/*Añadimos además el orden del producto y si está en el rango de fecha de estudio*/
drop table if exists sandbox.jpra_ventas_diarias;
create table sandbox.jpra_ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from sandbox.jpra_ventas base

inner join sandbox.jpra_calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join sandbox.jpra_articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo


group by 1,2,3,4,5,7,8);


/*Creamos una vista final con el filtrado para el estudio*/
drop view if exists sandbox.jpra_ventas_diarias_estudio_completo;
create view sandbox.jpra_ventas_diarias_estudio_completo as
(select *
from sandbox.jpra_ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Creamos una vista extra con el filtrado para el estudio que no tenga días de mayo 23 (que habrá que predecir)*/
drop view if exists sandbox.jpra_ventas_diarias_estudio;
create view sandbox.jpra_ventas_diarias_estudio as
(select *
from sandbox.jpra_ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [ ]:
# Crear la conexión
with engine.connect() as connection:
    # Establecer profundidad máxima de recursión para CTEs
    connection.execute(text("SET @@cte_max_recursion_depth = 3000;"))

    # Separar las consultas y limpiar
    consultas_lista = consulta_proceso.split(";")
    consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

    # Ejecutar cada consulta
    for consulta in consultas_lista:
        connection.execute(text(consulta))

    # Confirmar los cambios (si las consultas modifican datos)
    connection.commit()